Our project topic:
Back-test a rotation strategy based on aggregate valuations of the NIFTY non-financial sector indices (AUTO, COMMODITIES, FMCG, etc...)

How we have planned to go about it:

1. Taking 5-6 non-financial sector indices like Auto, commodities, metal, fmcg,pharma.

2. Determine seasonality and cyclicity in the each of these chosen indices. 

3. Based on this information, determine the frequency and the factors to be considered for rotation. 

4. Performing 5 different rotation strategies like:
Fixed ratio
Beta based
Variable ratio
Constant rupee plan
Constant ratio plan
This will help us understand the timing of entry and exit and the composition of the portfolio yielding maximum returns. 

5. Comparing the returns form each of these five types of strategies with that of a portfolio without any rotation IE, an untouched portfolio. 

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")


indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
today <- Sys.Date()
startDt <- today - 10

indices_data <-indices$NseTimeSeries() %>%
                  filter(TIME_STAMP >= startDt) %>%
                  arrange(desc(TIME_STAMP)) %>%
                  print(n=5)

# Source:     lazy query [?? x 7]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: desc(TIME_STAMP)
  NAME                 TIME_STAMP  HIGH   LOW  OPEN CLOSE    VOLUME
  <chr>                <date>     <dbl> <dbl> <dbl> <dbl>     <dbl>
1 NIFTY500 VALUE 50 TR 2019-08-30    NA    NA    NA 5142.        NA
2 NIFTY500 VALUE 50    2019-08-30    NA    NA    NA 3460. 362573360
3 NIFTY500 SHARIAH TR  2019-08-30    NA    NA    NA 3354.        NA
4 NIFTY500 SHARIAH     2019-08-30    NA    NA    NA 2858. 231565326
5 NIFTY50 VALUE 20 TR  2019-08-30    NA    NA    NA 6376.        NA
# … with more rows


In [3]:
id <- as.data.frame(indices_data)

In [4]:
id$CLOSE

[1]  5141.60  3460.25  3354.33  2857.76  6375.68  5229.95  5754.45   369.55
  [9]  2934.00  2463.23  4443.20   420.55 17977.47 11353.50    96.23 12013.72
 [17]  9322.40  3018.26  8518.65 10820.03  3849.52  3409.10  2328.58  2060.16
 [25] 16267.10 12200.15  2362.52  2089.53  8290.55  6871.14  8633.16  5469.60
 [33]  4401.48  6889.79  5445.95  1361.37  1346.97  3102.11  2625.25  5408.08
 [41]  4479.00  6668.11  4675.62  4020.91 21501.50 15816.85   290.09   267.20
 [49] 10168.92  8033.69 15309.20  3215.54  2474.40  4519.05  3028.95 17469.70
 [57]  9621.25  8072.05 36037.31 26405.05 18023.20 12361.35  6387.48  5272.35
 [65]  4353.23  5285.29  4261.55  6932.88  5712.70 20231.13 15652.20  3735.70
 [73]  3179.32  2289.95  2234.92  1914.65 11407.38  9368.78 12474.87  9769.24
 [81]  6712.11  5479.67 20040.31 16010.40  3657.35  3001.15  5532.77  1525.82
 [89]  1257.71  2035.10  2003.49  2113.66  2305.08  2130.76   956.90  1851.58
 [97]  6607.86  5636.90 43260.94 29257.35 15044.16 12418.15 22397.25 14382.25
[105]  3362.53  2488.30  2590.44  1846.30  4672.80  4197.64  3087.55 36861.20
[113] 27427.85  8731.44  7009.30  9439.99  7049.72 11328.89  9149.96 13888.65
[121] 11381.14 13815.05 11809.10  3052.08  2784.44 13449.30  8977.55 15440.47
[129] 11090.23  3770.90 11023.25  7115.27  5691.55  1830.07 14323.94 11155.10
[137]  5089.58  3425.24  3326.10  2833.71  6330.48  5192.85  5285.56  5677.50
[145]   372.05  2909.12  2442.34  4383.80   423.40 17849.10 11272.40    96.23
[153] 11917.01  9247.35  2988.16  8453.90 10737.76  3811.35  3375.30  2309.85
[161]  2043.59 16081.40 12060.90  2344.62  2073.70  8223.98  6815.96  8531.80
[169]  5399.41  4345.00  6808.90  5380.00  1353.83  1339.51  3060.48  2590.00
[177]  5359.64  4438.85  6587.34  4629.99  3981.67 21345.76 15702.30   286.43
[185]   263.80 10071.55  7956.76 15191.00  3183.10  2449.45  4539.62  3042.70
[193] 17334.82  9431.03  7912.50 35548.54 26046.90 17772.65 12189.50  6328.94
[201]  5224.05  4281.42  5234.71  4220.80  6868.68  5659.80 20040.80 15504.95
[209]  3674.10  3129.90  2254.35  2208.62  1892.15 11365.31  9334.23 12354.90
[217]  9675.29  6655.21  5433.22 19882.77 15884.55  3672.07  3013.25  5464.91
[225]  1506.27  1241.60  2037.28  2006.64  2115.64  2311.42  2133.32   957.64
[233]  1853.01  6538.64  5577.85 42513.48 28751.80 14950.81 12341.10 22488.14
[241] 14440.65  3339.49  2471.25  2616.52  1864.90  4615.45  4182.78  3076.60
[249] 36696.35 27305.20  8682.70  6970.15  9380.33  7005.16 11229.58  9069.75
[257] 13802.71 11310.72 13673.22 11687.85  3025.37  2760.07 13342.20  8906.10
[265] 15335.45 11031.77  3751.62  1848.09 10948.30  7059.29  5646.75  1829.81
[273] 14213.79 11069.30  4669.69  5096.17  3429.68  3326.49  2834.13  6327.51
[281]  5190.45  5332.68  5780.75   368.70  2903.76  2437.84  4463.50   419.60
[289] 17878.79 11291.15    96.23 11910.84  9242.55  2990.23  8483.90 10775.90
[297]  3838.15  3399.05  2329.64  2061.10 16099.53 12074.50  2365.74  2092.38
[305]  8278.22  6860.92  8564.37  5426.96  4367.17  6834.89  5427.20  1365.70
[313]  1351.25  3085.79  2611.45  5404.67  4476.55  6645.15  4629.41  3981.17
[321] 21599.52 15888.95   285.19   262.70 10083.53  7966.23 15434.00  3271.32
[329]  2517.30  4502.98  3018.15 17612.09  9223.86  7738.65 35528.37 26032.15
[337] 17773.09 12189.80  6362.88  5252.45  4315.09  5255.16  4237.25  6895.92
[345]  5682.70 20110.84 15560.70  3703.00  3091.20  2226.45  2242.63  1921.30
[353] 11485.53  9432.97 12395.37  9709.88  6694.27  5465.31 19876.94 15879.90
[361]  3663.78  3006.45  5472.38  1524.75  1256.83  2034.86  2004.22  2112.93
[369]  2313.98  2132.61   956.27  1850.03  6579.08  5612.35 42606.90 28815.00
[377] 15234.45 12575.25 22494.93 14445.00  3330.77  2464.95  2581.64  1840.05
[385]  4621.80  4174.91  3070.85 37367.15 27804.30  8742.95  7018.55  9371.45
[393]  6998.53 11249.63  9085.94 13880.13 11374.16 13806.12 11801.80  3044.75
[401]  2777.75 13439.14  8970.90 15472.44 11118.59  3781.75  1847.66 11046.10
[409]  7110.57  5687.85 

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)